In [1]:
#auto-generated code to read csv file
import types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


body = client_898a3789eb53423e8b3dd654559a9bb5.get_object(Bucket='segmentingandclusteringneighborho-donotdelete-pr-vbpylohrf1tsbj',Key='CanadaPostalCodes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

#show first five rows of data
df_postalcodes = pd.read_csv(body)
df_postalcodes.head()


,Postal Code,Borough,Neighbourhood
0,M9Z,Not assigned,Not assigned
1,M9Y,Not assigned,Not assigned
2,M9X,Not assigned,Not assigned
3,M9W,Etobicoke,"Northwest, West Humber - Clairville"
4,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [2]:
#drop any rows where Borough is Not assigned
NA_borough = df_postalcodes[df_postalcodes['Borough'] == 'Not assigned'].index
df_postalcodes.drop(NA_borough, inplace=True)
df_postalcodes.head()

,Postal Code,Borough,Neighbourhood
3,M9W,Etobicoke,"Northwest, West Humber - Clairville"
4,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
7,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
8,M9P,Etobicoke,Westmount
9,M9N,York,Weston


In [3]:
#split each neighborhood onto a separate line and reset the index
df_postalcodes = (df_postalcodes.set_index(['Postal Code', 'Borough'])
   .stack()
   .str.split(',', expand=True)
   .stack()
   .unstack(-2)
   .reset_index(-1, drop=True)
   .reset_index()
)
df_postalcodes.head(15)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,Malvern
1,M1B,Scarborough,Rouge
2,M1C,Scarborough,Rouge Hill
3,M1C,Scarborough,Port Union
4,M1C,Scarborough,Highland Creek
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [4]:
#show the final size of the dataframe
df_postalcodes.shape

(217, 3)

In [5]:
#auto-generated code to read new csv file
body = client_898a3789eb53423e8b3dd654559a9bb5.get_object(Bucket='segmentingandclusteringneighborho-donotdelete-pr-vbpylohrf1tsbj',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

#show first five rows of data
df_latlong = pd.read_csv(body)
df_latlong.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
#merge the 2 dataframes based on 'Postal Code' column
merged_Frame = pd.merge(df_postalcodes, df_latlong, on = 'Postal Code')
merged_Frame.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,43.806686,-79.194353
1,M1B,Scarborough,Rouge,43.806686,-79.194353
2,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
3,M1C,Scarborough,Port Union,43.784535,-79.160497
4,M1C,Scarborough,Highland Creek,43.784535,-79.160497


In [7]:
toronto_data = merged_Frame[merged_Frame['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West,43.679557,-79.352188
2,M4K,East Toronto,Riverdale,43.679557,-79.352188
3,M4L,East Toronto,India Bazaar,43.668999,-79.315572
4,M4L,East Toronto,The Beaches West,43.668999,-79.315572


In [8]:
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium

print('Libraries imported!')

     |████████████████████████████████| 93 kB 3.2 MB/s  eta 0:00:01
Libraries imported!


In [9]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


In [10]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)

map_Toronto

In [11]:
CLIENT_ID = 'XRFBDU3T2ZBWNGWDZ4OTI5YD1QFYN31CZGD3N2S4LSBJ5W5Q'
CLIENT_SECRET = 'K50H3XM12XQOILGD4WUNZYUUX13WB0HZLQSC1KQEIPJCCSTO' 
VERSION = '20180605'
LIMIT = 100
radius = 500

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude'])

The Beaches
The Danforth West
 Riverdale
India Bazaar
 The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
 Lawrence Park
Davisville
Moore Park
 Summerhill East
Summerhill West
 Rathnelly
 South Hill
 Forest Hill SE
 Deer Park
Rosedale
St. James Town
 Cabbagetown
Church and Wellesley
Regent Park
 Harbourfront
Garden District
 Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond
 Adelaide
 King
Harbourfront East
 Union Station
 Toronto Islands
Toronto Dominion Centre
 Design Exchange
Commerce Court
 Victoria Hotel
Roselawn
Forest Hill North & West
 Forest Hill Road Park
The Annex
 North Midtown
 Yorkville
University of Toronto
 Harbord
Kensington Market
 Chinatown
 Grange Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport
Stn A PO Boxes
First Canadian Place
 Underground city
Christie
Dufferin
 Dovercourt Village
Little Portugal
 Trinity
Brockton
 Parkdale Village
 Exhibition Place
High 

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(3174, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [15]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,97,97,97,97,97,97
Bathurst Quay,16,16,16,16,16,16
Cabbagetown,46,46,46,46,46,46
Chinatown,63,63,63,63,63,63
Deer Park,14,14,14,14,14,14
...,...,...,...,...,...,...
The Annex,19,19,19,19,19,19
The Beaches,6,6,6,6,6,6
The Danforth West,43,43,43,43,43,43


In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
toronto_onehot.shape

(3174, 236)

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Adelaide,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020619,0.0,...,0.000000,0.0,0.000000,0.00,0.010309,0.000000,0.000000,0.000000,0.0,0.000000
1,Bathurst Quay,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.0,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Cabbagetown,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.021739,0.0,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Chinatown,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.0,0.000000,0.00,0.063492,0.000000,0.047619,0.015873,0.0,0.000000
4,Deer Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.071429,0.0,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.071429,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,The Annex,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
73,The Beaches,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.000000,0.0,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
74,The Danforth West,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.023256,0.0,...,0.023256,0.0,0.023256,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.023256
75,Toronto Dominion Centre,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.0,...,0.000000,0.0,0.000000,0.01,0.010000,0.000000,0.000000,0.010000,0.0,0.000000


In [20]:
toronto_grouped.shape

(77, 236)

In [21]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide----
           venue  freq
0    Coffee Shop  0.08
1           Café  0.05
2     Restaurant  0.04
3          Hotel  0.04
4  Deli / Bodega  0.03


---- Bathurst Quay----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4           Airport  0.06


---- Cabbagetown----
                venue  freq
0         Coffee Shop  0.09
1   Convenience Store  0.04
2                Café  0.04
3  Italian Restaurant  0.04
4          Restaurant  0.04


---- Chinatown----
                           venue  freq
0                           Café  0.08
1  Vegetarian / Vegan Restaurant  0.06
2                    Coffee Shop  0.05
3          Vietnamese Restaurant  0.05
4             Mexican Restaurant  0.05


---- Deer Park----
                   venue  freq
0            Coffee Shop  0.14
1    Fried Chicken Joint  0.07
2       Sushi Restaurant  0.07
3                   Bank  0.07
4  Vietnamese Restaurant  0.07


---- Des

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega,Gym,Thai Restaurant,Concert Hall,Lounge,Salad Place
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane
2,Cabbagetown,Coffee Shop,Park,Café,Restaurant,Pub,Bakery,Pizza Place,Convenience Store,Italian Restaurant,Japanese Restaurant
3,Chinatown,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Bar,Arts & Crafts Store,Bakery,Dessert Shop,Gaming Cafe
4,Deer Park,Coffee Shop,Pizza Place,Pub,Restaurant,Bank,Supermarket,Sushi Restaurant,Bagel Shop,Liquor Store,Fried Chicken Joint


In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 0, 0, 4, 0], dtype=int32)

In [25]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Pub,Asian Restaurant,Health Food Store,Coffee Shop,Trail,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,M4K,East Toronto,The Danforth West,43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Pub,Caribbean Restaurant,Restaurant,Café
2,M4K,East Toronto,Riverdale,43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Pub,Caribbean Restaurant,Restaurant,Café
3,M4L,East Toronto,India Bazaar,43.668999,-79.315572,0,Fast Food Restaurant,Sandwich Place,Sushi Restaurant,Steakhouse,Brewery,Italian Restaurant,Pub,Fish & Chips Shop,Movie Theater,Ice Cream Shop
4,M4L,East Toronto,The Beaches West,43.668999,-79.315572,0,Fast Food Restaurant,Sandwich Place,Sushi Restaurant,Steakhouse,Brewery,Italian Restaurant,Pub,Fish & Chips Shop,Movie Theater,Ice Cream Shop


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
#cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Neighborhood,Pub,Asian Restaurant,Health Food Store,Coffee Shop,Trail,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,The Danforth West,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Pub,Caribbean Restaurant,Restaurant,Café
2,Riverdale,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Pub,Caribbean Restaurant,Restaurant,Café
3,India Bazaar,0,Fast Food Restaurant,Sandwich Place,Sushi Restaurant,Steakhouse,Brewery,Italian Restaurant,Pub,Fish & Chips Shop,Movie Theater,Ice Cream Shop
4,The Beaches West,0,Fast Food Restaurant,Sandwich Place,Sushi Restaurant,Steakhouse,Brewery,Italian Restaurant,Pub,Fish & Chips Shop,Movie Theater,Ice Cream Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
73,Swansea,0,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Pub,Diner,Bookstore,School,Sandwich Place,Burrito Place
74,Queen's Park,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,College Auditorium
75,Ontario Provincial Government,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,College Auditorium
76,Business reply mail Processing Centre,0,Light Rail Station,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park


In [28]:
#cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,CN Tower,1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane
51,King and Spadina,1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane
52,Railway Lands,1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane
53,Harbourfront West,1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane
54,Bathurst Quay,1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane
55,South Niagara,1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane
56,Island airport,1,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane


In [29]:
#cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Moore Park,2,Restaurant,Park,Tennis Court,Trail,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop
12,Summerhill East,2,Restaurant,Park,Tennis Court,Trail,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop
18,Rosedale,2,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


In [30]:
#cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Roselawn,3,Ice Cream Shop,Garden,Pool,Music Venue,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Donut Shop


In [31]:
#cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Lawrence Park,4,Park,Bus Line,Swim School,Business Service,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
40,Forest Hill North & West,4,Jewelry Store,Park,Sushi Restaurant,Bus Line,Trail,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
41,Forest Hill Road Park,4,Jewelry Store,Park,Sushi Restaurant,Bus Line,Trail,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
